#1. Preparação

In [ ]:
pip install geopandas

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#Outras
import pandas as pd
import geopandas as gpd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 2. Carregando as bases

In [ ]:
dtype = {'ANO_ELEICAO':'str', 'NR_TURNO':'str', 'SG_UF':'str', 'CD_MUNICIPIO':'str', 'NM_MUNICIPIO':'str',
       'NR_ZONA':'str', 'NR_LOCAL_VOTACAO':'str', 'CD_CARGO_PERGUNTA':'str', 'DS_CARGO_PERGUNTA':'str',
       'NR_PARTIDO':'str', 'SG_PARTIDO':'str', 'NM_PARTIDO':'str', 'CD_TIPO_VOTAVEL':'str',
       'DS_TIPO_VOTAVEL':'str', 'NR_VOTAVEL':'str', 'NM_VOTAVEL':'str', 'local_unico':'str',
       'QT_VOTOS':'int'}

In [ ]:
votos = pd.read_csv('/content/gdrive/MyDrive/dados/data/votos/boletim-urna-consolidado/2020_71110_geral.csv', sep=';', dtype=dtype)

In [ ]:
votos.head(3)

,ANO_ELEICAO,NR_TURNO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,local_unico,QT_VOTOS
0,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,-1,#NULO#,#NULO#,2,Branco,95,Branco,71072-1-1015,134
1,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,-1,#NULO#,#NULO#,3,Nulo,96,Nulo,71072-1-1015,221
2,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,10,REPUBLICANOS,REPUBLICANOS,1,Nominal,10,CELSO RUSSOMANNO,71072-1-1015,381


In [ ]:
local = gpd.read_file('/content/gdrive/MyDrive/dados/data/voronois/final/71072.shp')

In [ ]:
local.head(3)

,SGL_UF,COD_LOCALI,COD_TSE,LOCALIDADE,ZONA,NUM_LOCAL,LOCAL_VOTA,ENDERECO,BAIRRO_LOC,CEP,COUNTRY,id_unico,lat,lon,geom_str,CD_MUN,geometry
0,SP,3550308,71072,SÃO PAULO,1,1015,COLÉGIO OBJETIVO,"AV. PAULISTA, 900",BELA VISTA,01310100,BRASIL,71072-1-1015,-23.5653,-46.6510,"-23.5653, -46.6510",3550308,"POLYGON ((-46.65530 -23.56865, -46.65576 -23.5..."
1,SP,3550308,71072,SÃO PAULO,1,1023,EE. RODRIGUES ALVES,"AV. PAULISTA, 227",BELA VISTA,01311000,BRASIL,71072-1-1023,-23.5700,-46.6465,"-23.5700, -46.6465",3550308,"POLYGON ((-46.64608 -23.57233, -46.64810 -23.5..."
2,SP,3550308,71072,SÃO PAULO,1,1040,EMEF. CELSO LEITE RIBEIRO FILHO,"RUA HUMAITÁ, 480",BELA VISTA,01321010,BRASIL,71072-1-1040,-23.5573,-46.6418,"-23.5573, -46.6418",3550308,"POLYGON ((-46.64468 -23.55824, -46.64441 -23.5..."


In [ ]:
local.id_unico.nunique()

1999

In [ ]:
votos_geocode = pd.merge(votos, local[['id_unico']], left_on='local_unico', right_on='id_unico', how='left', indicator=True)

In [ ]:
#Calcular a perda de votos pelso locais que não foram georeferenciados
votos_perdidos = votos_geocode[votos_geocode['_merge'] == 'left_only']['QT_VOTOS'].sum()
perc_perdidos = votos_perdidos / votos['QT_VOTOS'].sum() * 100

print(votos_perdidos) #votos perdidos
print(votos['QT_VOTOS'].sum()) #total de votos
print(perc_perdidos) #Percental de votos perdidos

224358
12708200
1.7654585228435185


In [ ]:
#Selecionar somente os locais georeferenciados
votos_locais = votos_geocode[votos_geocode['_merge'] == 'both']
votos_locais.drop(columns=['_merge', 'id_unico'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
votos_locais

,ANO_ELEICAO,NR_TURNO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,local_unico,QT_VOTOS
0,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,-1,#NULO#,#NULO#,2,Branco,95,Branco,71072-1-1015,134
1,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,-1,#NULO#,#NULO#,3,Nulo,96,Nulo,71072-1-1015,221
2,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,10,REPUBLICANOS,REPUBLICANOS,1,Nominal,10,CELSO RUSSOMANNO,71072-1-1015,381
3,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,13,PT,Partido dos Trabalhadores,1,Nominal,13,JILMAR TATTO,71072-1-1015,115
4,2020,1,SP,71072,SÃO PAULO,1,1015,11,Prefeito,16,PSTU,Partido Socialista dos Trabalhadores Unificado,1,Nominal,16,VERA,71072-1-1015,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818072,2020,1,SP,71072,SÃO PAULO,6,2097,13,Vereador,90,PROS,Partido Republicano da Ordem Social,1,Nominal,90038,MAJOR HORTA,71072-6-2097,1
818073,2020,1,SP,71072,SÃO PAULO,6,2097,13,Vereador,90,PROS,Partido Republicano da Ordem Social,1,Nominal,90123,NENEM,71072-6-2097,1
818074,2020,1,SP,71072,SÃO PAULO,6,2097,13,Vereador,90,PROS,Partido Republicano da Ordem Social,1,Nominal,90444,ROBSON TRINDADE,71072-6-2097,1
818075,2020,1,SP,71072,SÃO PAULO,6,2097,13,Vereador,90,PROS,Partido Republicano da Ordem Social,1,Nominal,90600,RAFINHA,71072-6-2097,1


In [ ]:
votos_locais.to_csv('/content/gdrive/MyDrive/dados/data/votos/votos_final/2020_71110.csv')